In [ ]:
import pandas as pd
import numpy as np
import ast
from numpy import asarray

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
movies_metadata = pd.read_csv("movies_nonnull.csv")
movies_metadata.head()

In [ ]:
movies_metadata.info()

In [ ]:
remove_feature = ['Unnamed: 0','adult','id','video']

In [ ]:
keywords = pd.read_csv('D:/Ultimate USA/Data Science/Data analytics/Movie recommendation system/archive (4)/keywords.csv')
keywords.head()

In [ ]:
keywords.info()

In [ ]:
credits = pd.read_csv('D:/Ultimate USA/Data Science/Data analytics/Movie recommendation system/archive (4)/credits.csv')
credits.head()

In [ ]:
credits.info()

In [ ]:
merged_df = pd.merge(movies_metadata, keywords, on='id')
merged_df.head()

In [ ]:
movies_df = pd.merge(merged_df, credits, on='id')

In [ ]:
movies_df = movies_df.iloc[:8000,:]

In [ ]:
movies_df.drop(columns=['Unnamed: 0', 'adult', 'video'], inplace=True)

In [ ]:
movies_df = movies_df.set_index('id')

In [ ]:
(movies_df['revenue'] == 0.0).sum()

In [ ]:
(movies_df['budget'] == 0.0).sum()

In [ ]:
movies_df.drop(columns=['revenue', 'budget'], inplace=True)

In [ ]:
(movies_df['vote_count'] == 0.0).sum()

In [ ]:
movies_df.info()

In [ ]:
def extract_from_genre(x):
        t = []
        for j in ast.literal_eval(x):
            try:
                t.append(j['name'])
            except:
                pass
        return t
movies_df['genres'] = movies_df['genres'].apply(extract_from_genre)
movies_df['genres']

In [ ]:
def clean_lang(x):
    
    try:
        float(x)
        return 'Unknown'
    except:
        return x

movies_df['original_language'] = movies_df['original_language'].apply(clean_lang)
movies_df['original_language'].unique()

In [ ]:
import hashlib
hash_size = len(movies_df['original_language'].unique())  # number of hash values to use
movies_df['original_language'] = movies_df['original_language'].apply(lambda x: int(hashlib.sha256(x.encode('utf-8')).hexdigest(), 16) % hash_size)

In [ ]:
movies_df['original_language'].unique()

In [ ]:
def extract_from_original_title(x):
    t = []
    for i in x.split():
        t.append(i)
    
    return t
movies_df['original_title'] = movies_df['original_title'].apply(extract_from_original_title)
movies_df['original_title']

In [ ]:
def extract_from_overview(x):
    t = []
    for i in x.split(","):
        for j in i.split():
            t.append(j)
    
    return t
movies_df['overview'] = movies_df['overview'].apply(extract_from_overview)
movies_df['overview']

In [ ]:
def extract_from_production_companies(x):
        t = []
        for j in ast.literal_eval(x):
            try:
                t.append(j['name'])
            except:
                pass
        return t
movies_df['production_companies'] = movies_df['production_companies'].apply(extract_from_production_companies)
movies_df['production_companies']

In [ ]:
def extract_from_production_countries(x):
        t = []
        for j in ast.literal_eval(x):
            try:
                t.append(j['name'])
            except:
                pass
        return t
movies_df['production_countries'] = movies_df['production_countries'].apply(extract_from_production_countries)
movies_df['production_countries']

In [ ]:
def extract_from_spoken_lan(x):
        t = []
        for j in ast.literal_eval(x):
            try:
                t.append(j['name'])
            except:
                pass
        return t
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(extract_from_spoken_lan)
movies_df['spoken_languages']

In [ ]:
movies_df.join(pd.get_dummies(movies_df['status']))
movies_df.drop(columns=['status'], inplace=True)

In [ ]:
movies_df

In [ ]:
def extract_from_tagline(x):
    t = []
    for i in x.split():
        t.append(i)
    
    return t
movies_df['tagline'] = movies_df['tagline'].apply(extract_from_tagline)
movies_df['tagline']

In [ ]:
def extract_from_keywords(x):
        t = []
        for j in ast.literal_eval(x):
            try:
                t.append(j['name'])
            except:
                pass
        return t
movies_df['keywords'] = movies_df['keywords'].apply(extract_from_keywords)
movies_df['keywords']

In [ ]:
def extract_from_cast(x):
        t = []
        for j in ast.literal_eval(x):
            try:
                t.append(j['name'])
            except:
                pass
        return t
movies_df['cast'] = movies_df['cast'].apply(extract_from_keywords)
movies_df['cast']

In [ ]:
# def extract_from_crew(x):
#         t = []
#         for j in ast.literal_eval(x):
#             try:
#                 t.append(j['name'])
#             except:
#                 pass
#         return t
# movies_df['crew'] = movies_df['crew'].apply(extract_from_crew)
# movies_df['crew']

In [ ]:
del movies_df['crew']

In [ ]:
movies_df.iloc[1]['title']

In [ ]:
import re

def combine_text():
    columns = ['original_title','overview','spoken_languages','tagline','keywords', 'cast']
    final = []
    for i in range(len(movies_df)):
        t = ""
        for col in columns:
            for m in movies_df.iloc[i][col]:
                t += ' '+m.lower()
            
        final.append(t)
    return final

movies_df['metadata'] = pd.Series(combine_text())

In [ ]:
combined = combine_text()

In [ ]:
pd.Series(movies_df.index)

In [ ]:
new_df = pd.DataFrame(pd.Series(movies_df.index).values,pd.Series(combined))

In [ ]:
new_df = new_df.reset_index()

In [ ]:
new_df = new_df.rename(columns={0:'id','index':'metadata'})

In [ ]:
new_df

In [ ]:
del movies_df['metadata']

In [ ]:
movies_df = pd.merge(new_df, movies_df, on='id', how='inner')
movies_df.drop(columns=['original_title','overview','spoken_languages','tagline','keywords', 'cast'], inplace=True)

In [ ]:
movies_df.set_index('id', inplace=True)

In [ ]:
movies_df.info()

In [ ]:
movies_df = movies_df.loc[~movies_df.index.duplicated(keep='first')]

In [ ]:
del movies_df['title']

In [ ]:
copy_mdf = movies_df.copy()

In [ ]:
movies_df.head()

In [ ]:
movies_df = movies_df.reset_index()

In [ ]:
del movies_df['id']

In [ ]:
movies_df['original_language']

In [ ]:
copy_df = movies_df.copy()

In [ ]:
movies_df = copy_df.copy()

In [ ]:
movies_df = copy_mdf.copy()

In [ ]:
unique_genre_set = set()
def unique_genre():
    for i in movies_df['genres'].values:
        for j in i:
            unique_genre_set.add(j)
unique_genre()
def convert_genres():

    hash_size = len(unique_genre_set)  # number of hash values to use
    def convert_to_hash(x):
        t = []
 
        for i in x:
            t.append(int(hashlib.sha256(i.encode('utf-8')).hexdigest(), 16) % hash_size)
        return t
    
    movies_df['genres'] = movies_df['genres'].apply(convert_to_hash)
    max_length = 0
    final = []
    for i in movies_df['genres'].values:
        if len(i) > max_length:
            max_length = len(i)
    for j,i in enumerate(movies_df['genres'].values):
        final.append(np.pad(i, (0, max_length - len(i)), mode = 'constant', constant_values=0.0))
    return final
te = convert_genres()
te

In [ ]:
unique_prod_company_set = set()
def unique_prod_compny():
    for i in movies_df['production_companies'].values:
        for j in i:
            unique_prod_company_set.add(j)
unique_prod_compny()

def convert_prod_comapny():

    hash_size = len(unique_prod_company_set)  # number of hash values to use
    print(hash_size)
    def convert_to_hash(x):
        t = []
 
        for i in x:
            t.append(int(hashlib.sha256(i.encode('utf-8')).hexdigest(), 16) % hash_size)
        return t
    
    movies_df['production_companies'] = movies_df['production_companies'].apply(convert_to_hash)
    max_length = 0
    final = []
    for i in movies_df['production_companies'].values:
        if len(i) > max_length:
            max_length = len(i)
    print(max_length)
    for j,i in enumerate(movies_df['production_companies'].values):
        final.append(np.pad(i, (0, max_length - len(i)), mode = 'constant', constant_values=0.0))
    return final
te_prod_comapny = convert_prod_comapny()
te_prod_comapny

In [ ]:
prod_comapny_df = pd.DataFrame(np.array(te_prod_comapny))

In [ ]:
prod_comapny_df.head()

In [ ]:
unique_prod_country_set = set()
def unique_prod_country():
    for i in movies_df['production_countries'].values:
        for j in i:
            unique_prod_country_set.add(j)
unique_prod_country()

def convert_prod_country():

    hash_size = len(unique_prod_country_set)  # number of hash values to use
    print(hash_size)
    def convert_to_hash(x):
        t = []
 
        for i in x:
            t.append(int(hashlib.sha256(i.encode('utf-8')).hexdigest(), 16) % hash_size)
        return t
    
    movies_df['production_countries'] = movies_df['production_countries'].apply(convert_to_hash)
    max_length = 0
    final = []
    for i in movies_df['production_countries'].values:
        if len(i) > max_length:
            max_length = len(i)
    print(max_length)
    for j,i in enumerate(movies_df['production_countries'].values):
        final.append(np.pad(i, (0, max_length - len(i)), mode = 'constant', constant_values=0.0))
    return final
te_prod_country = convert_prod_country()


In [ ]:
prod_country_df = pd.DataFrame(np.array(te_prod_country))

In [ ]:
prod_country_df.head()

In [ ]:
from tqdm.notebook import tqdm
def convert_to_string():
    list_to_convert = ['genres']
    for i in list_to_convert:
        final = []
        for j in tqdm(movies_df[i].values):
            t = ""
            for k in j:
                t += ' '+k
            final.append(t)
        movies_df[i] = pd.Series(final)
    
convert_to_string()
    

In [ ]:
import datetime
def format_datetime(x):
    return datetime.datetime.strptime(x.split()[0], '%Y-%m-%d').date().year
movies_df['release_date'] = movies_df['release_date'].apply(format_datetime)

In [ ]:
movies_df
del movies_df['title']

In [ ]:
movies_df

In [ ]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

In [ ]:
vector = model.get_vector('apple')
print(vector.shape)

In [ ]:
from tqdm.notebook import tqdm
import itertools
import pickle
def convert_to_vector_metadata():
    final = []
    for i in movies_df['metadata'].values:
        t = []
        for j in i.split()[:100]:
            try:
                t.append(model.get_vector(j.lower()))
            except:
                pass
        final.append(t)
    max_length = 0
    for i in final:
        if len(i) > max_length:
            max_length = len(i)

        
    max_length *= 300

    padded_sequences = []
    for i in final:

        padded_sequences.append((np.pad(np.array(i).flatten(), (0, max_length - len(np.array(i).flatten())), mode='constant', constant_values=0.0)))
        
        

    return padded_sequences


temp = convert_to_vector_metadata()


In [ ]:
np.asarray(temp).shape

In [ ]:
metadta_df = pd.DataFrame(np.asarray(temp))

In [ ]:
metadta_df.head()

In [ ]:


metadta_df.head()

In [ ]:
genre_df.head()

In [ ]:
prod_comapny_df.head()

In [ ]:

prod_country_df.head()

In [ ]:
movies_df.drop(coalumns=['metadata','genres','production_companies','production_countries'], inplace=True)

In [ ]:
movies_df = movies_df.reset_index().drop(columns=['id'])

In [ ]:
movies_df

In [ ]:
merge_1 = pd.merge(movies_df, metadta_df, left_index=True, right_index=True)

In [ ]:
merge_2 = pd.merge(merge_1, genre_df, left_index=True, right_index=True)

In [ ]:
merge_3 = pd.merge(merge_2, prod_comapny_df, left_index=True, right_index=True)

In [ ]:
merge_4 = pd.merge(merge_3, prod_country_df, left_index=True, right_index=True)

In [ ]:
movies_df = merge_1

In [ ]:
pd.mergmovies_df

In [ ]:
final_df = pd.concat([movies_df, genre_df, prod_comapny_df,prod_country_df], axis=1)

In [ ]:
final_df.to_csv('final_df.csv')

In [ ]:
import pandas as pd
final_df = pd.read_csv('final_df.csv')

In [ ]:
new_columns = [f'new_col{i}' for i in range(len(final_df.columns))]

# use the rename method with a dictionary comprehension to change the column names
final_df = final_df.rename(columns={final_df.columns[i]: new_columns[i] for i in range(len(final_df.columns))})

In [ ]:
final_df.head()

In [ ]:
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=6,init='k-means++',max_iter=500).fit(final_df)

In [ ]:
final_df['KmeansCluster']=kmeans.labels_

print("Cluster 0: ",final_df[final_df['KmeansCluster']==0].KmeansCluster.count())
print("Cluster 1: ",final_df[final_df['KmeansCluster']==1].KmeansCluster.count())
print("Cluster 2: ",final_df[final_df['KmeansCluster']==2].KmeansCluster.count())
print("Cluster 3: ",final_df[final_df['KmeansCluster']==3].KmeansCluster.count())
print("Cluster 4: ",final_df[final_df['KmeansCluster']==4].KmeansCluster.count())
print("Cluster 5: ",final_df[final_df['KmeansCluster']==5].KmeansCluster.count())

In [ ]:
movies_pred = pd.read_csv("movies_nonnull.csv")
movies_pred = movies_pred.iloc[:8000,:]

In [ ]:
movies_pred = movies_pred.reset_index()

In [ ]:
movies_pred.drop(columns=['index','id'], inplace=True)

In [ ]:
movies_pred[movies_pred['title']== 'Independence Day']

In [ ]:
movies_pred.head()

In [ ]:
final_df.head()

In [ ]:
kmeans.predict(final_df.iloc[760,:-1].to_numpy().reshape(1,-1))

In [ ]:
final_df.iloc[475,:-1]

In [ ]:
pred = (final_df[final_df['KmeansCluster']==3].index.to_list())

In [ ]:
for i in pred[:10]:
    print(movies_pred.iloc[i]['title'])